In [ ]:
'''
This file processes 18 Limma files (DEGs at each of 9 timepoints for high dose, DEGs at each of 9 timepoints for low dose),
and 2 files from the output of DynGENIE3 (alphas above the median ~0.004 in high dosed and low dosed genes). 

It adds KEGG IDs to both files, sorts the files by their scores (logFC and alphas, respectively), and filters so 10k genes
or less are in each file.

Alphas tend to have a lot of duplicate scores. Jitter is added and scores are aggregated for duplicate genes. Unique values of 
genes and scores must be used for Gestalt.

'''

In [1]:
import pandas as pd
import numpy as np
import re
import glob

In [2]:
#import original gene expression files
expression_levels = pd.read_csv('rna_vst_proc.csv')
#import DEGs from DynGENIE3
control_only = pd.read_csv('control_only_genes_8.csv')
high_only = pd.read_csv('high_only_genes_8.csv')
low_only = pd.read_csv('low_only_genes_8.csv')
dosed_genes = pd.read_csv('dosed_genes_8.csv')
growth_reproduction_genes = pd.read_csv('growth_reproduction_genes_8.csv')
#import DEGs from Limma
data_dir = "limma_results"
tsv_files = glob.glob(f"{data_dir}/*.tabular")
limma_dfs = [pd.read_csv(file, sep="\t") for file in tsv_files]
#import human orthologs
human_orthologs = pd.read_csv('../ortholog/dma_hsa.tsv',sep='\t')

/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_89088/3531834859.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160) have mixed types. Specify dtype option on import or set low_memory=False.
  expression_levels = pd.read_csv('rna_vst_proc.csv')


In [12]:
#get kegg IDs and orthologs, matched on Dapma gene
human_renamed = human_orthologs.rename(columns={"Daphnia_magna": "treatment"})
expression_levels_renamed = expression_levels.rename(columns={"Unnamed: 0": "KEGG"})
kegg_human = pd.merge(expression_levels_renamed,human_renamed,on='treatment')

In [13]:
#function to prepare limma files for gestalt
def limma_gestalt(df,file_name):
    
    #rename kegg_human['target'] to GeneID
    kegg_human_renamed = kegg_human.rename(columns={"treatment":"GeneID"})
    kegg_human_subset = kegg_human_renamed[['KEGG','GeneID']]
    #drop any values where KEGG ID is not found
    kegg_human_subset = kegg_human_subset.dropna(subset=['KEGG'])
    #clean human orthologs so only one entry for one gene
    def retain_first_gene(gene_string):
        return gene_string.split(';')[0]

    # Apply the function to the 'genes' column
    kegg_human_subset['KEGG'] = kegg_human_subset['KEGG'].apply(retain_first_gene)
    #add kegg IDs on geneID
    merged_kegg_limma = pd.merge(kegg_human_subset,df,on='GeneID')
    #subset df to only two columns - KEGG and logfc
    kegg_limma_subset = merged_kegg_limma[['KEGG','logFC']]
    #order by absolute value of logfc
    limma_sorted = kegg_limma_subset.reindex(kegg_limma_subset['logFC'].abs().sort_values(ascending=False).index)
    #get top 10k genes and save rnk for Gestalt
    limma_head = limma_sorted.head(10000)
    #rename columns
    limma_head_renamed = limma_head.rename(columns={"KEGG":"GeneID","logFC":"score"})
    #save to rnk file
    limma_head_renamed.to_csv(file_name, sep='\t', index=False,header=False)

    return limma_head

In [14]:
#create file names for each condition/timepoint combination
file_name_list = ['limma_high_control_12H.rnk','limma_high_control_1H.rnk','limma_high_control_24H.rnk',
                'limma_high_control_2H.rnk','limma_high_control_4D.rnk','limma_high_control_5D.rnk','limma_high_control_6D.rnk',
                'limma_high_control_6H.rnk', 'limma_high_control_7D.rnk','limma_low_control_12H.rnk','limma_low_control_1H.rnk',
                'limma_low_control_24H.rnk','limma_low_control_2H.rnk','limma_low_control_4D.rnk','limma_low_control_5D.rnk',
                'limma_low_control_6D.rnk','limma_low_control_6H.rnk','limma_low_control_7D.rnk']

#run gestalt prep for 18 limma files
for file in range(len(file_name_list)):
    limma_gestalt(limma_dfs[file],file_name_list[file])

In [19]:
#prepare output of DynGENIE3 for Gestalt
def process_gestalt(df): 
    
    #process files for GESTALT, requires target genes and their alphas
    target = df[['target','alpha']]
    
    # Rename the 'target' column to 'regulatory' in control_target
    target_renamed = target.rename(columns={'target': 'treatment'})
    
    #add KEGG IDs and remove NANs
    kegg_gene_file = expression_levels_renamed[['KEGG','treatment']]
    merged_keggs = pd.merge(kegg_gene_file,target_renamed,on='treatment')
    
    # Sort by 'alpha' column
    sorted_df = merged_keggs.sort_values(by='alpha',ascending=False).reset_index(drop=True)

    #drop nans and remove semi-colons from kegg ids, rename to prepare for GMT
    nans_dropped = sorted_df.dropna(subset=['KEGG'])
    filtered_df = nans_dropped[['KEGG','alpha']]
    filtered_df['KEGG'] = filtered_df['KEGG'].str.split(';').str[0]
    renamed_df = filtered_df.rename(columns={'KEGG': 'NAME'})

    return renamed_df

In [25]:
#run initial processing for 3 conditions
ranked_genes_control = process_gestalt(control_only)
ranked_genes_low = process_gestalt(low_only)
ranked_genes_high = process_gestalt(high_only)

/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_89088/264906219.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['KEGG'] = filtered_df['KEGG'].str.split(';').str[0]
/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_89088/264906219.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['KEGG'] = filtered_df['KEGG'].str.split(';').str[0]
/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_89088/264906219.py:19: SettingWithCopyWarning: 
A value is trying to 

,NAME,alpha
101,K08852,0.978789
102,K08852,0.978789
103,K08852,0.978789
104,K08852,0.978789
105,K08852,0.978789


In [26]:
#add jitter and aggregate scores - Gestalt cannot handle duplicate scores or genes
def prepare_gestalt(df, file_name):
    
    # Add jitter to scores to handle ties
    np.random.seed(42)  # For reproducibility
    df['alpha'] = df['alpha'] + np.random.uniform(-0.01, 0.01, df.shape[0])

    # Aggregate scores for duplicated genes
    aggregated_scores = df.groupby('NAME')['alpha'].mean().reset_index()

    # Save to .rnk file
    aggregated_scores.to_csv(file_name, sep="\t", index=False, header=False)

# Prepare and save the .rnk files
prepare_gestalt(ranked_genes_control, 'gestalt_alphas_control.rnk')
prepare_gestalt(ranked_genes_low, 'gestalt_alphas_low.rnk')
prepare_gestalt(ranked_genes_high, 'gestalt_alphas_high.rnk')